SnapshotFunc

Files required:  
snapshotclass.ipynb in VICO_loc/  
zeusmp_snapshot_ready.py in VICO_loc/  
folder with name of snapshot in VICO_loc/Data/ (e.g. VICO_loc/Data/Snap004_n)  

Read in each snapshot e.g. if your snapshot data is stored in ./Data/fluxbug/04-1.4msun/ you would call,
read_snap('Snap004_n', 4, path=VICO_loc+'/Data/fluxbug/04-1.4msun/', read_zeusmp=True, calculate_intensity_vars=True, calculate_flux=True)

after the initial read in, if you have to restart partway through, you can then set read_zeusmp=False since this will be done at the beginning.



In [ ]:
# import io, os, sys, inspect, types

# currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# parentdir = os.path.dirname(currentdir)
# sys.path.insert(0, parentdir)
# from zeusmp_snapshot_reader import read_zeusmp_snapshot
# from zeusmp_snapshot_reader import ScaleFactors
import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams/')

from zeusmp_snapshot_reader import read_zeusmp_snapshot
from zeusmp_snapshot_reader import ScaleFactors
from snapshot import snapshot 
from basic_snapshot import basic_snapshot


import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from datetime import datetime


# !pip install ipynb
# from ipynb.fs.full.snapshotclass import *

# set this to be your own folder location
VICO_loc = '/Users/emigardiner/VICO/pjams-ionization'


In [ ]:
# INPUTS
frequencies = np.array([.01, .05, .1, .5, 1, 5.3, 23, 43, 100, 230 ]) # GHz
frequencies *= 10**9 # Hz
r_kpc = 1
scales = np.array([500,1000,2000,4000,8000,16000,32000]) # AU
heights = np.array([1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000,
                   10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000,
                   18000, 19000, 20000, 21000, 22000, 23000, 24000, 25000])
np.savez((VICO_loc+'/Data/heights_and_scales.npz'), scales=scales, heights=heights) 

In [ ]:
def plot_intensity_read_flux(snapshot, frequencies=frequencies, r_kpc=r_kpc, scales=scales):
    Fluxes = np.zeros((3, len(frequencies)))
    snapshot.load_shock_variables()

    scalefluxfile = open((VICO_loc+'/RadioEmissions/FluxOutputs/scale_fluxes_'+snapshot.name+'.txt'), 'a')
    heightfluxfile = open((VICO_loc+'/RadioEmissions/FluxOutputs/height_fluxdensities_'+snapshot.name+'.txt'), 'a')
    ScaleFluxes = np.zeros((3, len(frequencies), len(scales)))
    zScaleFluxes = np.zeros((2, len(frequencies), len(scales)))
    HeightFluxDensities = np.zeros((2, len(frequencies), len(heights)))
    
    for f in range(len(frequencies)):
        # snapshot.make_intensity_plots(nu=frequencies[f], r_kpc=r_kpc) # also loads intensity variables
        snapshot.load_intensity_variables(nu=frequencies[f])
        Fluxes[0,f] = snapshot.FnuA
        Fluxes[1,f] = snapshot.FnuB
        Fluxes[2,f] = snapshot.FnuC


        print(snapshot.name, 'Output at', datetime.now(), file=scalefluxfile)
        print('\n', snapshot.name, 'Output at', datetime.now(), file=scalefluxfile)
        print('nu (GHz) \tFluxA (mJy) \tFluxB (mJy) \tFluxC (mJy) \tx1 range (AU) \tx2 range (AU), \tx3 range (AU)', file=scalefluxfile)
        for s in range(len(scales)):
            FA, FB, FC, zFA, zFB = snapshot.calculate_scale_fluxes(scales[s], file=scalefluxfile, nu = frequencies[f], 
                                                                   r_kpc=1, printToScreen=False)
            ScaleFluxes[0,f,s] = FA
            ScaleFluxes[1,f,s] = FB
            ScaleFluxes[2,f,s] = FC
            zScaleFluxes[0,f,s] = zFA
            zScaleFluxes[1,f,s] = zFB
        for h in range(len(heights)):
            hFA = snapshot.calculate_height_fluxes(heights[h], file=heightfluxfile, nu = frequencies[f], 
                                                              r_kpc=1, width=1000, printToScreen=False)
            HeightFluxDensities[0,f,h] = hFA
                           
    scalefluxfile.close()
    np.savez((VICO_loc+'/Data/'+snapshot.name+'/'+snapshot.name+'_ScaleFluxes.npz'), 
             ScaleFluxes = ScaleFluxes, zScaleFluxes = zScaleFluxes, HeightFluxDensities = HeightFluxDensities)
    return ScaleFluxes, zScaleFluxes, HeightFluxDensities

In [ ]:
def read_snap(snapname, snapnum, path = VICO_loc+'/Data/', read_zeusmp=False, calculate_intensity_vars=False, calculate_flux=False):
    Snap = snapshot(snap=snapnum, name = snapname, path = path, read_zeusmp = read_zeusmp) 
    if(read_zeusmp):
        Snap.calculate_all_shock_variables()
        Snap.save_shock_variables()
    else:
        Snap.load_shock_variables()
    if(calculate_intensity_vars):
#         Snap.calculate_all_intensity_variables(nu=5.3 * 10**9, r_kpc=1)
#         Snap.calculate_all_intensity_variables(nu=23 * 10**9, r_kpc=1)
#         Snap.calculate_all_intensity_variables(nu=43 * 10**9, r_kpc=1)
#         Snap.calculate_all_intensity_variables(nu=100 * 10**9, r_kpc=1)
#         Snap.calculate_all_intensity_variables(nu=230 * 10**9, r_kpc=1)
#         Snap.calculate_all_intensity_variables(nu=.01 * 10**9, r_kpc=1)
#         Snap.calculate_all_intensity_variables(nu=.05 * 10**9, r_kpc=1)
#         Snap.calculate_all_intensity_variables(nu=.1 * 10**9, r_kpc=1)
#         Snap.calculate_all_intensity_variables(nu=.5 * 10**9, r_kpc=1)
        Snap.calculate_all_intensity_variables(nu=1 * 10**9, r_kpc=1)
    if(calculate_flux):
        plot_intensity_read_flux(Snap)
        Snap.scale_fluxes_const(frequencies=frequencies, const=.01, scales=scales)
        Snap.scale_fluxes_const(frequencies=frequencies, const=False, scales=scales)

    

In [ ]:
read_snap('Snap068_nlr', 68, path=VICO_loc+'/Data/LOWRES/68_LR/', read_zeusmp=True, calculate_intensity_vars=True, calculate_flux=True)

In [ ]:
# read_snap('Snap009_n', 9, path=VICO_loc+'/Data/fluxbug/09-2msun/', read_zeusmp=False, calculate_intensity_vars=True, calculate_flux=True)

In [ ]:
# read_snap('Snap021_n', 21, path=VICO_loc+'/Data/fluxbug/21-4msun/', read_zeusmp=True, calculate_intensity_vars=True, calculate_flux=True)

In [ ]:
# read_snap('Snap054_n', 54, path=VICO_loc+'/Data/fluxbug/54-12msun/', read_zeusmp=False, calculate_intensity_vars=True, calculate_flux=True)

In [ ]:
# read_snap('Snap068_n', 68, path=VICO_loc+'/Data/fluxbug/68-16msun/', read_zeusmp=True, calculate_intensity_vars=True, calculate_flux=True)

In [ ]:
# read_snap('Snap094_n', 94, path=VICO_loc+'/Data/fluxbug/94-24msun/', read_zeusmp=False, calculate_intensity_vars=True, calculate_flux=True)

In [ ]:
# read_snap('Snap106_n94', 106, path=VICO_loc+'/Data/fluxbug/94hf/', read_zeusmp=True, calculate_intensity_vars=True, calculate_flux=True)

In [ ]:
# read_snap('Snap107_n94', 107, path=VICO_loc+'/Data/fluxbug/94hf/', read_zeusmp=True, calculate_intensity_vars=True, calculate_flux=True)

In [ ]:
# read_snap('Snap108_n94', 108, path=VICO_loc+'/Data/fluxbug/94hf/', read_zeusmp=True, calculate_intensity_vars=True, calculate_flux=True)

In [ ]:
# read_snap('Snap055_n54', 55, path=VICO_loc+'/Data/fluxbug/54hf/', read_zeusmp=True, calculate_intensity_vars=True, calculate_flux=True)

In [ ]:
# read_snap('Snap056_n54', 56, path=VICO_loc+'/Data/fluxbug/54hf/', read_zeusmp=True, calculate_intensity_vars=True, calculate_flux=True)

In [ ]:
# read_snap('Snap057_n54', 57, path=VICO_loc+'/Data/fluxbug/54hf/', read_zeusmp=True, calculate_intensity_vars=True, calculate_flux=True)

In [ ]:
# read_snap('Snap058_n54', 58, path=VICO_loc+'/Data/fluxbug/54hf/', read_zeusmp=True, calculate_intensity_vars=True, calculate_flux=True)

In [ ]:
# read_snap('Snap006_n04', 6, path=VICO_loc+'/Data/fluxbug/04hf/', read_zeusmp=True, calculate_intensity_vars=True, calculate_flux=True)

In [ ]:
# read_snap('Snap116_n94', 116, path=VICO_loc+'/Data/fluxbug/94hf/', read_zeusmp=False, calculate_intensity_vars=True, calculate_flux=True)